# Route Efficiency Prediction Model - Refactored

**Objective**: Train a predictive model on `route_efficiency` using trip-level data + aggregated telemetry from rows.csv

**Key improvements**:
- Uses all three CSVs (trips, rows aggregates, combined if useful)
- Strict leakage controls (no target components as features)
- Explainability-first feature design
- API-ready artifacts for Flask/FastAPI deployment

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.cluster import KMeans
from sklearn.inspection import permutation_importance
import lightgbm as lgb
import joblib
import time
import os
import warnings
warnings.filterwarnings('ignore')

print("✓ Libraries imported")

## 1. Load and Audit Data

In [ ]:
# Load trips (base table: one row per trip)
trips = pd.read_csv('data/safetruck_data_iter1_trips.csv')

# Load rows (telemetry: many rows per trip)
rows = pd.read_csv('data/safetruck_data_iter1_rows.csv')

print("=" * 80)
print("DATA AUDIT")
print("=" * 80)
print(f"Trips: {len(trips):,} rows, {len(trips.columns)} columns")
print(f"  Columns: {list(trips.columns)}")
print(f"\nRows (telemetry): {len(rows):,} rows, {len(rows.columns)} columns")
print(f"  Columns: {list(rows.columns)}")
print(f"\nUnique trips in trips.csv: {trips['trip_id'].nunique():,}")
print(f"Unique trips in rows.csv: {rows['trip_id'].nunique():,}")
print(f"Unique vehicles in trips: {trips['vehicle_id'].nunique():,}")
print(f"Unique vehicles in rows: {rows['vehicle_id'].nunique():,}")
print("=" * 80)

## 2. Aggregate Row-Level Telemetry (Per Trip)

In [ ]:
%%time
# Compute robust telemetry aggregates per trip_id
# These capture driving behavior without directly encoding the target

print("Aggregating row-level telemetry...")

# Speed profile aggregates
speed_aggs = rows.groupby('trip_id')['speed_kmh'].agg([
    ('speed_mean', 'mean'),
    ('speed_median', 'median'),
    ('speed_p25', lambda x: x.quantile(0.25)),
    ('speed_p75', lambda x: x.quantile(0.75)),
    ('speed_p90', lambda x: x.quantile(0.90)),
    ('speed_std', 'std'),
    ('speed_min', 'min'),
    ('speed_max', 'max')
]).reset_index()

# Speed variability
speed_aggs['speed_iqr'] = speed_aggs['speed_p75'] - speed_aggs['speed_p25']
speed_aggs['speed_cv'] = speed_aggs['speed_std'] / (speed_aggs['speed_mean'] + 1e-6)

# Stop/idle behavior (speed < 3 km/h threshold)
stop_threshold = 3.0
rows['is_stopped'] = rows['speed_kmh'] < stop_threshold

stop_aggs = rows.groupby('trip_id').agg(
    stop_count=('is_stopped', 'sum'),
    total_rows=('is_stopped', 'count')
).reset_index()

# Overspeeding (speed > 90 km/h as example threshold)
overspeed_threshold = 90.0
rows['is_overspeeding'] = rows['speed_kmh'] > overspeed_threshold
overspeed_aggs = rows.groupby('trip_id')['is_overspeeding'].sum().reset_index()
overspeed_aggs.columns = ['trip_id', 'overspeed_count']

# Idle behavior (from is_idle flag)
idle_aggs = rows.groupby('trip_id')['is_idle'].agg([
    ('idle_event_count', 'sum'),
    ('idle_percentage_rows', 'mean')
]).reset_index()

# Moving behavior
moving_aggs = rows.groupby('trip_id')['is_moving'].agg([
    ('moving_event_count', 'sum'),
    ('moving_percentage_rows', 'mean')
]).reset_index()

# Merge all aggregates
telemetry_aggs = speed_aggs
telemetry_aggs = telemetry_aggs.merge(stop_aggs, on='trip_id', how='left')
telemetry_aggs = telemetry_aggs.merge(overspeed_aggs, on='trip_id', how='left')
telemetry_aggs = telemetry_aggs.merge(idle_aggs, on='trip_id', how='left')
telemetry_aggs = telemetry_aggs.merge(moving_aggs, on='trip_id', how='left')

# Fill NaN with 0 for counts
telemetry_aggs = telemetry_aggs.fillna(0)

print(f"✓ Aggregated {len(telemetry_aggs):,} trips")
print(f"  Aggregated features: {len(telemetry_aggs.columns) - 1}")
print(telemetry_aggs.head())

## 3. Merge Trips + Telemetry Aggregates

In [ ]:
# Join trips with telemetry aggregates
df = trips.merge(telemetry_aggs, on='trip_id', how='left')

# Normalize telemetry features per distance (per 100km) and per time (per hour)
epsilon = 1e-6

# Per 100km rates
df['stop_count_per_100km'] = df['stop_count'] / (df['Trip_Distance_km'] / 100 + epsilon)
df['overspeed_count_per_100km'] = df['overspeed_count'] / (df['Trip_Distance_km'] / 100 + epsilon)
df['idle_events_per_100km'] = df['idle_event_count'] / (df['Trip_Distance_km'] / 100 + epsilon)

# Per hour rates
df['stop_count_per_hour'] = df['stop_count'] / (df['Trip_Duration_min'] / 60 + epsilon)
df['overspeed_count_per_hour'] = df['overspeed_count'] / (df['Trip_Duration_min'] / 60 + epsilon)

print(f"✓ Combined dataset: {len(df):,} rows, {len(df.columns)} columns")
print(f"\nSample of normalized features:")
print(df[['trip_id', 'Trip_Distance_km', 'stop_count', 'stop_count_per_100km', 'overspeed_count_per_100km']].head())

## 4. QA Filtering

In [ ]:
# Convert timestamps
df['timestamp_start'] = pd.to_datetime(df['timestamp_start'])
df['timestamp_end'] = pd.to_datetime(df['timestamp_end'])

# Apply strict QA filters
print(f"Initial trips: {len(df):,}")

df_clean = df[
    (df['QA_zero_duration'] == False) &
    (df['QA_negative_distance'] == False) &
    (df['Trip_Duration_min'] > 0) &
    (df['Trip_Distance_km'] > 0) &
    (df['lat_start'].notna()) &
    (df['lon_start'].notna()) &
    (df['lat_end'].notna()) &
    (df['lon_end'].notna())
].copy()

print(f"After QA filters: {len(df_clean):,}")
print(f"Removed: {len(df) - len(df_clean):,} trips ({100*(len(df) - len(df_clean))/len(df):.1f}%)")
print("✓ QA filtering complete")

## 5. Feature Engineering (Leakage-Safe)

In [ ]:
def haversine(lat1, lon1, lat2, lon2):
    """Calculate great-circle distance in km"""
    R = 6371  # Earth radius in km
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

def calculate_bearing(lat1, lon1, lat2, lon2):
    """Calculate bearing in degrees"""
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    x = np.sin(dlon) * np.cos(lat2)
    y = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(dlon)
    return np.degrees(np.arctan2(x, y))

# Geographic features
df_clean['haversine_km'] = haversine(
    df_clean['lat_start'], df_clean['lon_start'],
    df_clean['lat_end'], df_clean['lon_end']
)
df_clean['bearing'] = calculate_bearing(
    df_clean['lat_start'], df_clean['lon_start'],
    df_clean['lat_end'], df_clean['lon_end']
)
df_clean['delta_lat'] = df_clean['lat_end'] - df_clean['lat_start']
df_clean['delta_lon'] = df_clean['lon_end'] - df_clean['lon_start']

# Time features (cyclical encoding)
df_clean['hour_of_day'] = df_clean['timestamp_start'].dt.hour
df_clean['day_of_week'] = df_clean['timestamp_start'].dt.dayofweek
df_clean['hour_sin'] = np.sin(2 * np.pi * df_clean['hour_of_day'] / 24)
df_clean['hour_cos'] = np.cos(2 * np.pi * df_clean['hour_of_day'] / 24)
df_clean['dow_sin'] = np.sin(2 * np.pi * df_clean['day_of_week'] / 7)
df_clean['dow_cos'] = np.cos(2 * np.pi * df_clean['day_of_week'] / 7)

# Distance bucket
df_clean['distance_bucket'] = pd.cut(
    df_clean['Trip_Distance_km'],
    bins=[0, 10, 50, 200, float('inf')],
    labels=['0-10km', '10-50km', '50-200km', '200+km']
)

print("✓ Geographic and time features engineered")
print(df_clean[['haversine_km', 'bearing', 'hour_sin', 'hour_cos', 'distance_bucket']].head())

## 6. Define Target Variable (route_efficiency)

**CRITICAL**: We compute route_efficiency as our target but exclude its components from features to avoid leakage

In [ ]:
# Use the pre-computed efficiency columns as components
# These exist in the trips CSV but will NOT be used as features

# Check if route_efficiency already exists in the data
if 'route_efficiency' not in df_clean.columns:
    # Compute composite efficiency (weighted combination)
    # directness = haversine / actual_distance
    # idle_efficiency = 1 - (idle_time / duration)
    # We'll use the provided efficiency columns if available
    
    directness_eff = np.clip(df_clean['haversine_km'] / (df_clean['Trip_Distance_km'] + epsilon), 0, 1)
    idle_eff = 1 - (df_clean['Idle_Time_min'] / (df_clean['Trip_Duration_min'] + epsilon))
    
    # Composite: 50% directness, 30% idle, 20% speed (relative to trip average)
    # For simplicity, normalize speed against median for that distance bucket
    df_clean['route_efficiency'] = (
        0.5 * directness_eff +
        0.3 * idle_eff +
        0.2 * np.clip(df_clean['Avg_Speed'] / 60, 0, 1)  # normalize to ~60 km/h
    )
else:
    print("Using existing route_efficiency column")

print(f"\nTarget variable (route_efficiency) distribution:")
print(df_clean['route_efficiency'].describe())
print(f"\nCorrelation with provided efficiency metrics:")
corr_cols = ['route_efficiency', 'Distance_Efficiency', 'Time_Efficiency', 'Idle_Efficiency']
print(df_clean[corr_cols].corr()['route_efficiency'])

## 7. Time-Based Train/Test Split (80/20)

In [ ]:
# Sort by time and split
df_clean = df_clean.sort_values('timestamp_start').reset_index(drop=True)

split_idx = int(len(df_clean) * 0.8)
train_data = df_clean.iloc[:split_idx].copy()
test_data = df_clean.iloc[split_idx:].copy()

print("=" * 80)
print("TRAIN/TEST SPLIT")
print("=" * 80)
print(f"Train: {len(train_data):,} trips")
print(f"  Period: {train_data['timestamp_start'].min()} to {train_data['timestamp_start'].max()}")
print(f"\nTest: {len(test_data):,} trips")
print(f"  Period: {test_data['timestamp_start'].min()} to {test_data['timestamp_start'].max()}")
print(f"\nVehicles in train: {train_data['vehicle_id'].nunique()}")
print(f"Vehicles in test: {test_data['vehicle_id'].nunique()}")
print(f"Overlap: {len(set(train_data['vehicle_id']) & set(test_data['vehicle_id']))}")
print("=" * 80)

## 8. Zone Clustering (Train-Fit Only)

In [ ]:
k = 50  # Number of zones

# Fit KMeans on training data only
print("Fitting KMeans for origin zones...")
kmeans_start = KMeans(n_clusters=k, random_state=42, n_init=10)
train_data['zone_start_id'] = kmeans_start.fit_predict(train_data[['lat_start', 'lon_start']])
test_data['zone_start_id'] = kmeans_start.predict(test_data[['lat_start', 'lon_start']])

print("Fitting KMeans for destination zones...")
kmeans_end = KMeans(n_clusters=k, random_state=42, n_init=10)
train_data['zone_end_id'] = kmeans_end.fit_predict(train_data[['lat_end', 'lon_end']])
test_data['zone_end_id'] = kmeans_end.predict(test_data[['lat_end', 'lon_end']])

print(f"✓ Created {k} origin zones and {k} destination zones")
print(f"  Train zone_start_id range: {train_data['zone_start_id'].min()}-{train_data['zone_start_id'].max()}")
print(f"  Train zone_end_id range: {train_data['zone_end_id'].min()}-{train_data['zone_end_id'].max()}")

## 9. Feature Selection (Leakage-Safe)

**Excluded to prevent leakage**:
- `Distance_Efficiency`, `Time_Efficiency`, `Idle_Efficiency` (direct target components)
- Any derived features that are exact transformations of the target formula

In [ ]:
# Numeric features (no leakage)
numeric_features = [
    # Trip metrics (raw inputs)
    'Trip_Distance_km', 'Trip_Duration_min', 'Avg_Speed',
    'Idle_Time_min', 'Moving_Time_min', 'Idle_Percentage',
    
    # Geographic
    'haversine_km', 'bearing', 'delta_lat', 'delta_lon',
    'lat_start', 'lon_start', 'lat_end', 'lon_end',
    
    # Time (cyclical)
    'hour_sin', 'hour_cos', 'dow_sin', 'dow_cos',
    
    # Vehicle metadata
    'Age_of_Truck_months',
    
    # Telemetry aggregates (from rows.csv)
    'speed_mean', 'speed_median', 'speed_p25', 'speed_p75', 'speed_p90',
    'speed_std', 'speed_iqr', 'speed_cv',
    'stop_count', 'stop_count_per_100km', 'stop_count_per_hour',
    'overspeed_count', 'overspeed_count_per_100km', 'overspeed_count_per_hour',
    'idle_event_count', 'idle_percentage_rows', 'idle_events_per_100km',
    'moving_event_count', 'moving_percentage_rows'
]

# Categorical features
categorical_features = ['vehicle_id', 'zone_start_id', 'zone_end_id', 'distance_bucket']

# Filter to only features that exist in both train and test
numeric_features = [f for f in numeric_features if f in train_data.columns and f in test_data.columns]
categorical_features = [f for f in categorical_features if f in train_data.columns and f in test_data.columns]

# Fill NaN in numeric features
for feat in numeric_features:
    train_data[feat] = train_data[feat].fillna(0)
    test_data[feat] = test_data[feat].fillna(0)

print("=" * 80)
print("FEATURE SELECTION")
print("=" * 80)
print(f"Numeric features ({len(numeric_features)}): ")
for f in numeric_features:
    print(f"  - {f}")
print(f"\nCategorical features ({len(categorical_features)}): {categorical_features}")
print("=" * 80)

## 10. Target Encoding for vehicle_id (Train-Fit Only)

In [ ]:
# Target encode vehicle_id on training data
vehicle_means = train_data.groupby('vehicle_id')['route_efficiency'].mean()
global_mean = train_data['route_efficiency'].mean()

# Apply to train and test
train_data['vehicle_id_encoded'] = train_data['vehicle_id'].map(vehicle_means).fillna(global_mean)
test_data['vehicle_id_encoded'] = test_data['vehicle_id'].map(vehicle_means).fillna(global_mean)

print(f"✓ Target encoded vehicle_id using {len(vehicle_means)} unique vehicles")
print(f"  Global mean route_efficiency: {global_mean:.4f}")
print(f"  Unknown vehicles in test will use global mean")

# Update feature lists
numeric_features.append('vehicle_id_encoded')
categorical_features.remove('vehicle_id')

## 11. Prepare X and y for Modeling

In [ ]:
# One-hot encode categorical features
train_encoded = pd.get_dummies(train_data, columns=categorical_features, prefix=categorical_features)
test_encoded = pd.get_dummies(test_data, columns=categorical_features, prefix=categorical_features)

# Get dummy column names
dummy_cols_train = [col for col in train_encoded.columns if any(col.startswith(f"{cat}_") for cat in categorical_features)]
dummy_cols_test = [col for col in test_encoded.columns if any(col.startswith(f"{cat}_") for cat in categorical_features)]

train_encoded = train_encoded[dummy_cols_train]
test_encoded = test_encoded[dummy_cols_test]

# Align test to train columns
test_encoded = test_encoded.reindex(columns=train_encoded.columns, fill_value=0)

# Combine numeric + categorical
X_train = pd.concat([
    train_data[numeric_features].reset_index(drop=True),
    train_encoded.reset_index(drop=True)
], axis=1)

X_test = pd.concat([
    test_data[numeric_features].reset_index(drop=True),
    test_encoded.reset_index(drop=True)
], axis=1)

y_train = train_data['route_efficiency'].values
y_test = test_data['route_efficiency'].values

print("=" * 80)
print("FEATURE MATRICES")
print("=" * 80)
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")
print(f"\nTotal features: {X_train.shape[1]}")
print(f"  Numeric: {len(numeric_features)}")
print(f"  Categorical (one-hot): {len(train_encoded.columns)}")
print("=" * 80)

## 12. Baseline Model: Ridge Regression

In [ ]:
# Scale for linear model
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train Ridge
ridge = Ridge(alpha=1.0, random_state=42)
ridge.fit(X_train_scaled, y_train)

# Predictions
y_train_pred_ridge = ridge.predict(X_train_scaled)
y_test_pred_ridge = ridge.predict(X_test_scaled)

# Evaluate
train_mae_ridge = mean_absolute_error(y_train, y_train_pred_ridge)
train_rmse_ridge = np.sqrt(mean_squared_error(y_train, y_train_pred_ridge))
train_r2_ridge = r2_score(y_train, y_train_pred_ridge)

test_mae_ridge = mean_absolute_error(y_test, y_test_pred_ridge)
test_rmse_ridge = np.sqrt(mean_squared_error(y_test, y_test_pred_ridge))
test_r2_ridge = r2_score(y_test, y_test_pred_ridge)

print("=" * 80)
print("RIDGE REGRESSION RESULTS")
print("=" * 80)
print(f"Train - MAE: {train_mae_ridge:.4f}, RMSE: {train_rmse_ridge:.4f}, R²: {train_r2_ridge:.4f}")
print(f"Test  - MAE: {test_mae_ridge:.4f}, RMSE: {test_rmse_ridge:.4f}, R²: {test_r2_ridge:.4f}")
print("=" * 80)

## 13. Main Model: LightGBM Regressor

In [ ]:
%%time
# Train LightGBM
lgbm = lgb.LGBMRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=7,
    num_leaves=31,
    min_child_samples=20,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    verbose=-1
)

lgbm.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    eval_metric='mae',
    callbacks=[lgb.early_stopping(stopping_rounds=30, verbose=False)]
)

# Predictions
y_train_pred_lgbm = lgbm.predict(X_train)
y_test_pred_lgbm = lgbm.predict(X_test)

# Evaluate
train_mae_lgbm = mean_absolute_error(y_train, y_train_pred_lgbm)
train_rmse_lgbm = np.sqrt(mean_squared_error(y_train, y_train_pred_lgbm))
train_r2_lgbm = r2_score(y_train, y_train_pred_lgbm)

test_mae_lgbm = mean_absolute_error(y_test, y_test_pred_lgbm)
test_rmse_lgbm = np.sqrt(mean_squared_error(y_test, y_test_pred_lgbm))
test_r2_lgbm = r2_score(y_test, y_test_pred_lgbm)

print("=" * 80)
print("LIGHTGBM RESULTS")
print("=" * 80)
print(f"Train - MAE: {train_mae_lgbm:.4f}, RMSE: {train_rmse_lgbm:.4f}, R²: {train_r2_lgbm:.4f}")
print(f"Test  - MAE: {test_mae_lgbm:.4f}, RMSE: {test_rmse_lgbm:.4f}, R²: {test_r2_lgbm:.4f}")
print(f"\nBest iteration: {lgbm.best_iteration_}")
print("=" * 80)

## 14. Feature Importance Analysis

In [ ]:
# Get LightGBM feature importances
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': lgbm.feature_importances_
}).sort_values('importance', ascending=False)

print("=" * 80)
print("TOP 30 MOST IMPORTANT FEATURES (LightGBM Gain)")
print("=" * 80)
print(feature_importance.head(30).to_string(index=False))
print("=" * 80)

# Identify top features by type
print("\nTop features by category:")
print("\n1. Top telemetry/behavior features:")
telemetry_feats = feature_importance[feature_importance['feature'].str.contains('speed|stop|idle|overspeed|moving')]
print(telemetry_feats.head(10).to_string(index=False))

print("\n2. Top geographic features:")
geo_feats = feature_importance[feature_importance['feature'].str.contains('lat|lon|haversine|bearing|delta|zone')]
print(geo_feats.head(10).to_string(index=False))

print("\n3. Top time features:")
time_feats = feature_importance[feature_importance['feature'].str.contains('hour|dow')]
print(time_feats.head(5).to_string(index=False))

## 15. Model Comparison Summary

In [ ]:
# Summary comparison
results = pd.DataFrame({
    'Model': ['Ridge Regression', 'LightGBM'],
    'Train MAE': [train_mae_ridge, train_mae_lgbm],
    'Test MAE': [test_mae_ridge, test_mae_lgbm],
    'Train RMSE': [train_rmse_ridge, train_rmse_lgbm],
    'Test RMSE': [test_rmse_ridge, test_rmse_lgbm],
    'Train R²': [train_r2_ridge, train_r2_lgbm],
    'Test R²': [test_r2_ridge, test_r2_lgbm]
})

print("\n" + "=" * 80)
print("MODEL COMPARISON SUMMARY")
print("=" * 80)
print(results.to_string(index=False))
print("=" * 80)
print(f"\nBest model by Test MAE: {results.loc[results['Test MAE'].idxmin(), 'Model']}")
print(f"Best model by Test R²: {results.loc[results['Test R²'].idxmax(), 'Model']}")
print(f"\nImprovement of LightGBM over Ridge:")
print(f"  MAE: {(test_mae_ridge - test_mae_lgbm) / test_mae_ridge * 100:.1f}% better")
print(f"  R²: {(test_r2_lgbm - test_r2_ridge) / (1 - test_r2_ridge) * 100:.1f}% variance explained improvement")

## 16. Package Model Artifacts for API Deployment

In [ ]:
# Create models directory if it doesn't exist
os.makedirs('models', exist_ok=True)

# Bundle all artifacts needed for inference
artifacts = {
    # Trained model
    "model": lgbm,
    
    # KMeans transformers for zone clustering
    "kmeans_start": kmeans_start,
    "kmeans_end": kmeans_end,
    
    # Target encoding for vehicle_id
    "vehicle_means": vehicle_means,
    "global_mean": float(global_mean),
    
    # Feature lists (CRITICAL: must match training order)
    "numeric_features": numeric_features.copy(),
    "dummy_columns": list(train_encoded.columns),
    
    # Distance bucket configuration
    "bucket_bins": [0, 10, 50, 200, float('inf')],
    "bucket_labels": ['0-10km', '10-50km', '50-200km', '200+km'],
    
    # Thresholds used in aggregation
    "stop_threshold_kmh": 3.0,
    "overspeed_threshold_kmh": 90.0,
    
    # Constants
    "epsilon": 1e-6,
    
    # Metadata
    "model_version": "lgbm_refactored_v2",
    "training_timestamp": time.time(),
    "train_size": len(train_data),
    "test_size": len(test_data),
    "test_mae": float(test_mae_lgbm),
    "test_rmse": float(test_rmse_lgbm),
    "test_r2": float(test_r2_lgbm),
    "feature_count": X_train.shape[1],
    "best_iteration": lgbm.best_iteration_
}

# Save artifacts
artifacts_path = 'models/model_artifacts.joblib'
joblib.dump(artifacts, artifacts_path)

print("=" * 80)
print("MODEL ARTIFACTS SAVED")
print("=" * 80)
print(f"Path: {artifacts_path}")
print(f"Model version: {artifacts['model_version']}")
print(f"Test MAE: {artifacts['test_mae']:.4f}")
print(f"Test RMSE: {artifacts['test_rmse']:.4f}")
print(f"Test R²: {artifacts['test_r2']:.4f}")
print(f"Total features: {artifacts['feature_count']}")
print(f"  Numeric: {len(artifacts['numeric_features'])}")
print(f"  Categorical (one-hot): {len(artifacts['dummy_columns'])}")
print(f"Unique vehicles: {len(artifacts['vehicle_means'])}")
print(f"Best iteration: {artifacts['best_iteration']}")
print("=" * 80)

## 17. API Input Requirements

In [ ]:
print("=" * 80)
print("REQUIRED API INPUTS FOR INFERENCE (POST-TRIP SCORING)")
print("=" * 80)
print("""
The Flask/FastAPI endpoint needs the following from the client:

REQUIRED FIELDS:
  - vehicle_id (string): Vehicle identifier
  - timestamp_start (ISO datetime): Trip start time
  - lat_start, lon_start (float): Origin coordinates
  - lat_end, lon_end (float): Destination coordinates
  - Trip_Distance_km (float): Total distance traveled
  - Trip_Duration_min (float): Total duration in minutes
  - Idle_Time_min (float): Idle time in minutes
  - Moving_Time_min (float): Moving time in minutes
  - Avg_Speed (float): Average speed in km/h
  - Idle_Percentage (float): Percentage of time idle (0-100)
  - Age_of_Truck_months (float): Age of vehicle in months

OPTIONAL (for richer predictions with telemetry):
  - If you can also provide row-level telemetry, compute aggregates:
    - speed_mean, speed_median, speed_p25, speed_p75, speed_p90
    - speed_std, speed_iqr, speed_cv
    - stop_count, overspeed_count
    - idle_event_count, moving_event_count
    - Normalized rates: *_per_100km, *_per_hour

RESPONSE:
  - predicted_route_efficiency (float, 0-1)
  - model_version (string)
  - Optional: top_features (list of {"feature": str, "contribution": float})
"""")
print("=" * 80)

## 18. Test Inference Pipeline (Simulate API Call)

In [ ]:
# Reload artifacts to verify they work
loaded_artifacts = joblib.load('models/model_artifacts.joblib')

print("=" * 80)
print("ARTIFACT LOADING VERIFICATION")
print("=" * 80)
print(f"✓ Model type: {type(loaded_artifacts['model']).__name__}")
print(f"✓ KMeans start clusters: {loaded_artifacts['kmeans_start'].n_clusters}")
print(f"✓ KMeans end clusters: {loaded_artifacts['kmeans_end'].n_clusters}")
print(f"✓ Vehicle means count: {len(loaded_artifacts['vehicle_means'])}")
print(f"✓ Global mean: {loaded_artifacts['global_mean']:.4f}")
print(f"✓ Numeric features: {len(loaded_artifacts['numeric_features'])}")
print(f"✓ Dummy columns: {len(loaded_artifacts['dummy_columns'])}")
print(f"✓ Model version: {loaded_artifacts['model_version']}")
print("=" * 80)

# Test prediction on a single sample from test set
sample_idx = 10
sample_row = test_data.iloc[sample_idx]

print(f"\nTEST PREDICTION ON SAMPLE (index {sample_idx}):")
print(f"Trip ID: {sample_row['trip_id']}")
print(f"Vehicle ID: {sample_row['vehicle_id']}")
print(f"Actual route_efficiency: {sample_row['route_efficiency']:.4f}")

# Get the corresponding X_test row
sample_X = X_test.iloc[sample_idx:sample_idx+1]
sample_pred = loaded_artifacts['model'].predict(sample_X)[0]

print(f"Predicted route_efficiency: {sample_pred:.4f}")
print(f"Absolute error: {abs(sample_pred - sample_row['route_efficiency']):.4f}")
print("=" * 80)
print("\n✓ All artifacts loaded successfully!")
print("✓ Model is ready for deployment in Flask/FastAPI")

# model training

In [1]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.cluster import KMeans
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

## 1. Load Data

In [2]:
# Load trips data (backbone) - TRIPS-ONLY PIPELINE
trips = pd.read_csv('data/safetruck_data_iter1_trips.csv')

print(f"Trips shape: {trips.shape}")
print("\nTrips columns:", trips.columns.tolist())

Trips shape: (145995, 22)

Trips columns: ['vehicle_id', 'trip_id', 'timestamp_start', 'timestamp_end', 'lat_start', 'lon_start', 'lat_end', 'lon_end', 'Trip_Distance_km', 'Trip_Duration_min', 'Avg_Speed', 'Idle_Time_min', 'Moving_Time_min', 'Idle_Percentage', 'Distance_Efficiency', 'Time_Efficiency', 'Idle_Efficiency', 'Signal_Reliability', 'Age_of_Truck_months', 'total_rows', 'QA_negative_distance', 'QA_zero_duration']


## 2. QA Filtering

In [3]:
# Convert timestamps to datetime
trips['timestamp_start'] = pd.to_datetime(trips['timestamp_start'])
trips['timestamp_end'] = pd.to_datetime(trips['timestamp_end'])

# Apply strict QA filters
print(f"Initial trips: {len(trips)}")

# Drop QA anomalies
trips_clean = trips[
    (trips['QA_zero_duration'] == False) &
    (trips['QA_negative_distance'] == False) &
    (trips['Trip_Duration_min'] > 0) &
    (trips['Trip_Distance_km'] > 0) &
    (trips['lat_start'].notna()) &
    (trips['lon_start'].notna()) &
    (trips['lat_end'].notna()) &
    (trips['lon_end'].notna())
].copy()

print(f"After QA filters: {len(trips_clean)}")
print(f"Removed {len(trips) - len(trips_clean)} trips ({100*(len(trips) - len(trips_clean))/len(trips):.1f}%)")

Initial trips: 145995
After QA filters: 55261
Removed 90734 trips (62.1%)


## 3. Geographic Features & Haversine Distance

In [4]:
def haversine(lat1, lon1, lat2, lon2):
    """Calculate great-circle distance in km using haversine formula"""
    R = 6371  # Earth radius in km
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

def calculate_bearing(lat1, lon1, lat2, lon2):
    """Calculate bearing from start to end point in degrees"""
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    x = np.sin(dlon) * np.cos(lat2)
    y = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(dlon)
    bearing = np.arctan2(x, y)
    return np.degrees(bearing)

# Calculate haversine distance
trips_clean['haversine_km'] = haversine(
    trips_clean['lat_start'], 
    trips_clean['lon_start'],
    trips_clean['lat_end'], 
    trips_clean['lon_end']
)

# Calculate detour ratio
epsilon = 1e-6
trips_clean['detour_ratio'] = trips_clean['Trip_Distance_km'] / (trips_clean['haversine_km'] + epsilon)

# Calculate directness efficiency (clipped to [0, 1])
trips_clean['directness_efficiency'] = np.clip(
    trips_clean['haversine_km'] / (trips_clean['Trip_Distance_km'] + epsilon), 
    0, 1
)

# Calculate bearing
trips_clean['bearing'] = calculate_bearing(
    trips_clean['lat_start'], 
    trips_clean['lon_start'],
    trips_clean['lat_end'], 
    trips_clean['lon_end']
)

# Delta lat/lon
trips_clean['delta_lat'] = trips_clean['lat_end'] - trips_clean['lat_start']
trips_clean['delta_lon'] = trips_clean['lon_end'] - trips_clean['lon_start']

print("Geographic features computed:")
print(trips_clean[['haversine_km', 'detour_ratio', 'directness_efficiency', 'bearing']].describe())

Geographic features computed:
       haversine_km  detour_ratio  directness_efficiency       bearing
count  55261.000000  5.526100e+04           55261.000000  55261.000000
mean     194.512867  2.470591e+04               0.225262     28.589770
std      393.672828  3.218665e+06               0.397734    103.461589
min        0.000000  8.680198e-07               0.000000   -179.977752
25%        2.415024  7.530712e+00               0.000329    -51.192373
50%      128.358150  8.945813e+02               0.001118     -0.022469
75%      300.611110  3.000000e+03               0.132790    140.479564
max    11555.312191  7.521470e+08               1.000000    180.000000


## 4. Time Features

In [5]:
# Extract time features
trips_clean['hour_of_day'] = trips_clean['timestamp_start'].dt.hour
trips_clean['day_of_week'] = trips_clean['timestamp_start'].dt.dayofweek

# Cyclical encoding for hour (0-23)
trips_clean['hour_sin'] = np.sin(2 * np.pi * trips_clean['hour_of_day'] / 24)
trips_clean['hour_cos'] = np.cos(2 * np.pi * trips_clean['hour_of_day'] / 24)

# Cyclical encoding for day of week (0-6)
trips_clean['dow_sin'] = np.sin(2 * np.pi * trips_clean['day_of_week'] / 7)
trips_clean['dow_cos'] = np.cos(2 * np.pi * trips_clean['day_of_week'] / 7)

print("Time features computed:")
print(trips_clean[['hour_of_day', 'day_of_week', 'hour_sin', 'hour_cos', 'dow_sin', 'dow_cos']].head())

Time features computed:
   hour_of_day  day_of_week  hour_sin  hour_cos   dow_sin   dow_cos
0            8            4  0.866025 -0.500000 -0.433884 -0.900969
1           11            4  0.258819 -0.965926 -0.433884 -0.900969
2           15            4 -0.707107 -0.707107 -0.433884 -0.900969
3           15            4 -0.707107 -0.707107 -0.433884 -0.900969
4           16            4 -0.866025 -0.500000 -0.433884 -0.900969


## 5. Operational Features

In [6]:
# Calculate operational ratios
trips_clean['idle_ratio'] = trips_clean['Idle_Time_min'] / (trips_clean['Trip_Duration_min'] + epsilon)
trips_clean['moving_ratio'] = trips_clean['Moving_Time_min'] / (trips_clean['Trip_Duration_min'] + epsilon)
trips_clean['idle_per_km'] = trips_clean['Idle_Time_min'] / (trips_clean['Trip_Distance_km'] + epsilon)

# Distance buckets for speed normalization
trips_clean['distance_bucket'] = pd.cut(
    trips_clean['Trip_Distance_km'],
    bins=[0, 10, 50, 200, float('inf')],
    labels=['0-10km', '10-50km', '50-200km', '200+km']
)

print("Operational features computed:")
print(trips_clean[['idle_ratio', 'moving_ratio', 'idle_per_km', 'distance_bucket']].describe())
print("\nDistance bucket distribution:")
print(trips_clean['distance_bucket'].value_counts().sort_index())

Operational features computed:
         idle_ratio  moving_ratio   idle_per_km
count  55261.000000  55261.000000  5.526100e+04
mean       0.422166      0.577759  5.633546e+00
std        0.363724      0.363754  9.136388e+01
min        0.000000      0.000000  0.000000e+00
25%        0.000000      0.285712  0.000000e+00
50%        0.370370      0.629628  2.196183e-07
75%        0.714284      0.999940  1.410396e-06
max        1.000000      1.000000  8.173638e+03

Distance bucket distribution:
distance_bucket
0-10km      13361
10-50km       448
50-200km      449
200+km      41003
Name: count, dtype: int64


## 6. Compute Target Variable: route_efficiency (Trips-Only)

In [7]:
# TRIPS-ONLY: Skip row-level and combined data joins
# All features will be derived directly from trips CSV
print("Using trips-only pipeline - no external joins needed")

Using trips-only pipeline - no external joins needed


## 7. Compute Target Variable: route_efficiency

In [8]:
# Time-based split first (to compute ref speeds on training data only)
trips_clean = trips_clean.sort_values('timestamp_start').reset_index(drop=True)

# 80-20 time split
split_idx = int(len(trips_clean) * 0.8)
train_data = trips_clean.iloc[:split_idx].copy()
test_data = trips_clean.iloc[split_idx:].copy()

print(f"Train set: {len(train_data)} trips (from {train_data['timestamp_start'].min()} to {train_data['timestamp_start'].max()})")
print(f"Test set: {len(test_data)} trips (from {test_data['timestamp_start'].min()} to {test_data['timestamp_start'].max()})")

# Compute reference speeds per distance bucket on TRAIN data only
ref_speeds = train_data.groupby('distance_bucket')['Avg_Speed'].median().to_dict()
print("\nReference speeds per bucket (from training data):")
for bucket, speed in ref_speeds.items():
    print(f"  {bucket}: {speed:.2f} km/h")

# Compute speed efficiency for both train and test
def compute_speed_efficiency(row):
    bucket = row['distance_bucket']
    ref_speed = ref_speeds.get(bucket, row['Avg_Speed'])  # fallback to own speed if bucket not in ref
    return np.clip(row['Avg_Speed'] / (ref_speed + epsilon), 0, 1)

train_data['speed_efficiency'] = train_data.apply(compute_speed_efficiency, axis=1)
test_data['speed_efficiency'] = test_data.apply(compute_speed_efficiency, axis=1)

# Idle efficiency
train_data['idle_efficiency'] = 1 - train_data['idle_ratio']
test_data['idle_efficiency'] = 1 - test_data['idle_ratio']

# Compute composite route_efficiency = 0.5*directness + 0.3*idle + 0.2*speed
train_data['route_efficiency'] = (
    0.5 * train_data['directness_efficiency'] +
    0.3 * train_data['idle_efficiency'] +
    0.2 * train_data['speed_efficiency']
)

test_data['route_efficiency'] = (
    0.5 * test_data['directness_efficiency'] +
    0.3 * test_data['idle_efficiency'] +
    0.2 * test_data['speed_efficiency']
)

print("\nTarget variable (route_efficiency) distribution:")
print(train_data['route_efficiency'].describe())
print(f"\nCorrelation with provided efficiencies (train):")
print(train_data[['route_efficiency', 'Distance_Efficiency', 'Time_Efficiency', 'Idle_Efficiency']].corr()['route_efficiency'])

Train set: 44208 trips (from 2025-08-01 00:00:17+00:00 to 2025-08-25 20:13:56+00:00)
Test set: 11053 trips (from 2025-08-25 20:14:04+00:00 to 2025-08-31 13:44:31+00:00)

Reference speeds per bucket (from training data):
  0-10km: 42.16 km/h
  10-50km: 22.03 km/h
  50-200km: 30.46 km/h
  200+km: 113646288.75 km/h

Target variable (route_efficiency) distribution:
count    44208.000000
mean         0.425302
std          0.269620
min          0.000051
25%          0.201379
50%          0.350117
75%          0.500452
max          1.000000
Name: route_efficiency, dtype: float64

Correlation with provided efficiencies (train):
route_efficiency       1.000000
Distance_Efficiency    0.872618
Time_Efficiency       -0.666498
Idle_Efficiency        0.638403
Name: route_efficiency, dtype: float64


## 8. Zone Clustering (k=50)

In [9]:
# Fit k-means on training data for start and end zones
k = 50

# Start zones
kmeans_start = KMeans(n_clusters=k, random_state=42, n_init=10)
train_data['zone_start_id'] = kmeans_start.fit_predict(train_data[['lat_start', 'lon_start']])
test_data['zone_start_id'] = kmeans_start.predict(test_data[['lat_start', 'lon_start']])

# End zones
kmeans_end = KMeans(n_clusters=k, random_state=42, n_init=10)
train_data['zone_end_id'] = kmeans_end.fit_predict(train_data[['lat_end', 'lon_end']])
test_data['zone_end_id'] = kmeans_end.predict(test_data[['lat_end', 'lon_end']])

print(f"Created {k} start zones and {k} end zones")
print(f"Train zone_start_id range: {train_data['zone_start_id'].min()}-{train_data['zone_start_id'].max()}")
print(f"Train zone_end_id range: {train_data['zone_end_id'].min()}-{train_data['zone_end_id'].max()}")

Created 50 start zones and 50 end zones
Train zone_start_id range: 0-49
Train zone_end_id range: 0-49


## 9. Prepare Feature Sets (Trips-Only)

In [10]:
# Define numerical features (trips-only, no row/combined aggregates)
numeric_features = [
    # Base trip metrics
    'Trip_Distance_km', 'Trip_Duration_min', 'Avg_Speed',
    'Idle_Time_min', 'Moving_Time_min', 'Idle_Percentage',
    # Geographic
    'haversine_km', 'detour_ratio', 'bearing', 'delta_lat', 'delta_lon',
    'lat_start', 'lon_start', 'lat_end', 'lon_end',
    # Time (cyclical)
    'hour_sin', 'hour_cos', 'dow_sin', 'dow_cos',
    # Operational
    'idle_ratio', 'moving_ratio', 'idle_per_km',
    # Metadata
    'Age_of_Truck_months'
]

# Categorical features
categorical_features = ['vehicle_id', 'zone_start_id', 'zone_end_id', 'distance_bucket']

# Filter to only features that exist in both train and test
numeric_features = [f for f in numeric_features if f in train_data.columns and f in test_data.columns]
categorical_features = [f for f in categorical_features if f in train_data.columns and f in test_data.columns]

print(f"Numerical features ({len(numeric_features)}): {numeric_features}")
print(f"\nCategorical features ({len(categorical_features)}): {categorical_features}")

# Fill NaN values in numeric features with 0 (should be minimal in trips-only)
for feat in numeric_features:
    train_data[feat] = train_data[feat].fillna(0)
    test_data[feat] = test_data[feat].fillna(0)

Numerical features (23): ['Trip_Distance_km', 'Trip_Duration_min', 'Avg_Speed', 'Idle_Time_min', 'Moving_Time_min', 'Idle_Percentage', 'haversine_km', 'detour_ratio', 'bearing', 'delta_lat', 'delta_lon', 'lat_start', 'lon_start', 'lat_end', 'lon_end', 'hour_sin', 'hour_cos', 'dow_sin', 'dow_cos', 'idle_ratio', 'moving_ratio', 'idle_per_km', 'Age_of_Truck_months']

Categorical features (4): ['vehicle_id', 'zone_start_id', 'zone_end_id', 'distance_bucket']


## 11. Target Encoding for vehicle_id

In [11]:
# Target encode vehicle_id using training data
vehicle_means = train_data.groupby('vehicle_id')['route_efficiency'].mean()
global_mean = train_data['route_efficiency'].mean()

# Apply to train and test
train_data['vehicle_id_encoded'] = train_data['vehicle_id'].map(vehicle_means).fillna(global_mean)
test_data['vehicle_id_encoded'] = test_data['vehicle_id'].map(vehicle_means).fillna(global_mean)

print(f"Target encoded vehicle_id using {len(vehicle_means)} unique vehicles")
print(f"Global mean route_efficiency: {global_mean:.4f}")

# Update feature lists
numeric_features.append('vehicle_id_encoded')
categorical_features.remove('vehicle_id')

Target encoded vehicle_id using 31 unique vehicles
Global mean route_efficiency: 0.4253


## 12. Prepare X and y for Modeling

In [12]:
# One-hot encode categorical features (zones and distance_bucket)
# Use columns= to force encoding of all categorical features (even if they're integers)
train_encoded = pd.get_dummies(train_data, columns=categorical_features, prefix=categorical_features)
test_encoded = pd.get_dummies(test_data, columns=categorical_features, prefix=categorical_features)

# Keep only the newly created dummy columns (remove original data columns)
dummy_cols_train = [col for col in train_encoded.columns if any(col.startswith(f"{cat}_") for cat in categorical_features)]
dummy_cols_test = [col for col in test_encoded.columns if any(col.startswith(f"{cat}_") for cat in categorical_features)]

train_encoded = train_encoded[dummy_cols_train]
test_encoded = test_encoded[dummy_cols_test]

# Align columns between train and test (test must have all train columns)
test_encoded = test_encoded.reindex(columns=train_encoded.columns, fill_value=0)

# Combine numeric and categorical
X_train = pd.concat([
    train_data[numeric_features].reset_index(drop=True),
    train_encoded.reset_index(drop=True)
], axis=1)

X_test = pd.concat([
    test_data[numeric_features].reset_index(drop=True),
    test_encoded.reset_index(drop=True)
], axis=1)

y_train = train_data['route_efficiency'].values
y_test = test_data['route_efficiency'].values

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")


X_train shape: (44208, 128)
X_test shape: (11053, 128)
y_train shape: (44208,)
y_test shape: (11053,)


## 13. Baseline Model: Ridge Regression

In [13]:
# Scale features for linear model
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train Ridge regression
ridge = Ridge(alpha=1.0, random_state=42)
ridge.fit(X_train_scaled, y_train)

# Predictions
y_train_pred_ridge = ridge.predict(X_train_scaled)
y_test_pred_ridge = ridge.predict(X_test_scaled)

# Evaluate
train_mae_ridge = mean_absolute_error(y_train, y_train_pred_ridge)
train_rmse_ridge = np.sqrt(mean_squared_error(y_train, y_train_pred_ridge))
train_r2_ridge = r2_score(y_train, y_train_pred_ridge)

test_mae_ridge = mean_absolute_error(y_test, y_test_pred_ridge)
test_rmse_ridge = np.sqrt(mean_squared_error(y_test, y_test_pred_ridge))
test_r2_ridge = r2_score(y_test, y_test_pred_ridge)

print("=" * 60)
print("RIDGE REGRESSION RESULTS")
print("=" * 60)
print(f"Train - MAE: {train_mae_ridge:.4f}, RMSE: {train_rmse_ridge:.4f}, R²: {train_r2_ridge:.4f}")
print(f"Test  - MAE: {test_mae_ridge:.4f}, RMSE: {test_rmse_ridge:.4f}, R²: {test_r2_ridge:.4f}")
print("=" * 60)

RIDGE REGRESSION RESULTS
Train - MAE: 0.0626, RMSE: 0.0895, R²: 0.8898
Test  - MAE: 0.0654, RMSE: 0.0933, R²: 0.8821


## 14. Main Model: LightGBM

In [14]:
# Train LightGBM (no scaling needed for tree models)
lgbm = lgb.LGBMRegressor(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=7,
    num_leaves=31,
    min_child_samples=20,
    random_state=42,
    verbose=-1
)

lgbm.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    eval_metric='mae',
    callbacks=[lgb.early_stopping(stopping_rounds=20, verbose=False)]
)

# Predictions
y_train_pred_lgbm = lgbm.predict(X_train)
y_test_pred_lgbm = lgbm.predict(X_test)

# Evaluate
train_mae_lgbm = mean_absolute_error(y_train, y_train_pred_lgbm)
train_rmse_lgbm = np.sqrt(mean_squared_error(y_train, y_train_pred_lgbm))
train_r2_lgbm = r2_score(y_train, y_train_pred_lgbm)

test_mae_lgbm = mean_absolute_error(y_test, y_test_pred_lgbm)
test_rmse_lgbm = np.sqrt(mean_squared_error(y_test, y_test_pred_lgbm))
test_r2_lgbm = r2_score(y_test, y_test_pred_lgbm)

print("=" * 60)
print("LIGHTGBM RESULTS")
print("=" * 60)
print(f"Train - MAE: {train_mae_lgbm:.4f}, RMSE: {train_rmse_lgbm:.4f}, R²: {train_r2_lgbm:.4f}")
print(f"Test  - MAE: {test_mae_lgbm:.4f}, RMSE: {test_rmse_lgbm:.4f}, R²: {test_r2_lgbm:.4f}")
print("=" * 60)

LIGHTGBM RESULTS
Train - MAE: 0.0023, RMSE: 0.0044, R²: 0.9997
Test  - MAE: 0.0026, RMSE: 0.0054, R²: 0.9996


## 15. Feature Importance Analysis

In [15]:
# Get feature importances from LightGBM
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': lgbm.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 20 Most Important Features:")
print(feature_importance.head(20).to_string(index=False))


Top 20 Most Important Features:
                 feature  importance
               Avg_Speed        1299
            detour_ratio        1129
             idle_per_km         946
         Idle_Percentage         467
              idle_ratio         430
            moving_ratio         258
            haversine_km         176
       Trip_Duration_min         168
  distance_bucket_200+km         156
        Trip_Distance_km         142
           Idle_Time_min         132
         Moving_Time_min         119
 distance_bucket_10-50km         118
distance_bucket_50-200km          81
      vehicle_id_encoded          48
               lon_start          36
               lat_start          30
               delta_lat          29
     Age_of_Truck_months          29
                hour_cos          26


## 16. Model Comparison Summary

In [16]:
# Summary comparison
results = pd.DataFrame({
    'Model': ['Ridge Regression', 'LightGBM'],
    'Train MAE': [train_mae_ridge, train_mae_lgbm],
    'Test MAE': [test_mae_ridge, test_mae_lgbm],
    'Train RMSE': [train_rmse_ridge, train_rmse_lgbm],
    'Test RMSE': [test_rmse_ridge, test_rmse_lgbm],
    'Train R²': [train_r2_ridge, train_r2_lgbm],
    'Test R²': [test_r2_ridge, test_r2_lgbm]
})

print("\n" + "=" * 80)
print("MODEL COMPARISON SUMMARY")
print("=" * 80)
print(results.to_string(index=False))
print("=" * 80)
print(f"\nBest model by Test MAE: {results.loc[results['Test MAE'].idxmin(), 'Model']}")
print(f"Best model by Test R²: {results.loc[results['Test R²'].idxmax(), 'Model']}")


MODEL COMPARISON SUMMARY
           Model  Train MAE  Test MAE  Train RMSE  Test RMSE  Train R²  Test R²
Ridge Regression   0.062578  0.065404    0.089511   0.093338  0.889779 0.882053
        LightGBM   0.002335  0.002622    0.004380   0.005449  0.999736 0.999598

Best model by Test MAE: LightGBM
Best model by Test R²: LightGBM


## 17. Package & Pickle Model Artifacts for Deployment

In [17]:
import joblib
import time
import os

# Create models directory if it doesn't exist
os.makedirs('models', exist_ok=True)

# Bundle all artifacts needed for inference
artifacts = {
    # Trained model
    "model": lgbm,
    
    # KMeans transformers for zone clustering
    "kmeans_start": kmeans_start,
    "kmeans_end": kmeans_end,
    
    # Target encoding for vehicle_id
    "vehicle_means": vehicle_means,
    "global_mean": float(global_mean),
    
    # Reference speeds per distance bucket (for versioning/audit, not needed in prediction)
    "ref_speeds": ref_speeds,
    
    # Feature lists (CRITICAL: must match training order)
    "numeric_features": numeric_features.copy(),
    "dummy_columns": list(train_encoded.columns),  # exact one-hot columns from training
    
    # Distance bucket configuration
    "bucket_bins": [0, 10, 50, 200, float('inf')],
    "bucket_labels": ['0-10km', '10-50km', '50-200km', '200+km'],
    
    # Constants
    "epsilon": float(epsilon),
    
    # Metadata
    "model_version": "lgbm_trips_only_v1",
    "training_timestamp": time.time(),
    "train_size": len(train_data),
    "test_size": len(test_data),
    "test_mae": float(test_mae_lgbm),
    "test_r2": float(test_r2_lgbm),
    "feature_count": X_train.shape[1]
}

# Save artifacts
artifacts_path = 'models/model_artifacts.joblib'
joblib.dump(artifacts, artifacts_path)

print("=" * 80)
print("MODEL ARTIFACTS SAVED")
print("=" * 80)
print(f"Path: {artifacts_path}")
print(f"Model version: {artifacts['model_version']}")
print(f"Test MAE: {artifacts['test_mae']:.4f}")
print(f"Test R²: {artifacts['test_r2']:.4f}")
print(f"Total features: {artifacts['feature_count']}")
print(f"Numeric features: {len(artifacts['numeric_features'])}")
print(f"Dummy columns: {len(artifacts['dummy_columns'])}")
print(f"Unique vehicles: {len(artifacts['vehicle_means'])}")
print("=" * 80)

# Show what's required from API payload
print("\nREQUIRED API INPUTS (for inference):")
required_inputs = [
    "vehicle_id",
    "timestamp_start",
    "lat_start", "lon_start", 
    "lat_end", "lon_end",
    "Trip_Distance_km",
    "Trip_Duration_min",
    "Idle_Time_min",
    "Moving_Time_min",
    "Avg_Speed",
    "Idle_Percentage",
    "Age_of_Truck_months"
]
for inp in required_inputs:
    print(f"  - {inp}")
    
print("\nNOTE: All engineered features (geo, time, operational, zones) will be")
print("      computed from these raw inputs at inference time using the saved")
print("      transformers and the same feature engineering logic.")

MODEL ARTIFACTS SAVED
Path: models/model_artifacts.joblib
Model version: lgbm_trips_only_v1
Test MAE: 0.0026
Test R²: 0.9996
Total features: 128
Numeric features: 24
Dummy columns: 104
Unique vehicles: 31

REQUIRED API INPUTS (for inference):
  - vehicle_id
  - timestamp_start
  - lat_start
  - lon_start
  - lat_end
  - lon_end
  - Trip_Distance_km
  - Trip_Duration_min
  - Idle_Time_min
  - Moving_Time_min
  - Avg_Speed
  - Idle_Percentage
  - Age_of_Truck_months

NOTE: All engineered features (geo, time, operational, zones) will be
      computed from these raw inputs at inference time using the saved
      transformers and the same feature engineering logic.


## 18. Verify Artifact Loading (Sanity Check)

In [18]:
# Reload artifacts to verify they can be loaded correctly
loaded_artifacts = joblib.load('models/model_artifacts.joblib')

print("=" * 80)
print("ARTIFACT LOADING VERIFICATION")
print("=" * 80)
print(f"✓ Model type: {type(loaded_artifacts['model']).__name__}")
print(f"✓ KMeans start clusters: {loaded_artifacts['kmeans_start'].n_clusters}")
print(f"✓ KMeans end clusters: {loaded_artifacts['kmeans_end'].n_clusters}")
print(f"✓ Vehicle means count: {len(loaded_artifacts['vehicle_means'])}")
print(f"✓ Global mean: {loaded_artifacts['global_mean']:.4f}")
print(f"✓ Numeric features: {len(loaded_artifacts['numeric_features'])}")
print(f"✓ Dummy columns: {len(loaded_artifacts['dummy_columns'])}")
print(f"✓ Distance buckets: {loaded_artifacts['bucket_labels']}")
print(f"✓ Model version: {loaded_artifacts['model_version']}")
print(f"✓ Training timestamp: {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(loaded_artifacts['training_timestamp']))}")
print("=" * 80)

# Test prediction on a single sample from test set
sample_idx = 0
sample_row = test_data.iloc[sample_idx]

print("\nTEST PREDICTION ON SAMPLE:")
print(f"Sample trip_id: {sample_row.get('trip_id', 'N/A')}")
print(f"Actual route_efficiency: {sample_row['route_efficiency']:.4f}")

# Get the corresponding X_test row
sample_X = X_test.iloc[sample_idx:sample_idx+1]
sample_pred = loaded_artifacts['model'].predict(sample_X)[0]

print(f"Predicted route_efficiency: {sample_pred:.4f}")
print(f"Absolute error: {abs(sample_pred - sample_row['route_efficiency']):.4f}")
print("=" * 80)

print("\n✓ All artifacts loaded successfully!")
print("✓ Model is ready for deployment in Flask API")

ARTIFACT LOADING VERIFICATION
✓ Model type: LGBMRegressor
✓ KMeans start clusters: 50
✓ KMeans end clusters: 50
✓ Vehicle means count: 31
✓ Global mean: 0.4253
✓ Numeric features: 24
✓ Dummy columns: 104
✓ Distance buckets: ['0-10km', '10-50km', '50-200km', '200+km']
✓ Model version: lgbm_trips_only_v1
✓ Training timestamp: 2025-10-24 02:52:49

TEST PREDICTION ON SAMPLE:
Sample trip_id: 118523
Actual route_efficiency: 0.4279
Predicted route_efficiency: 0.4328
Absolute error: 0.0050

✓ All artifacts loaded successfully!
✓ Model is ready for deployment in Flask API


## 19. Inference Example (How Flask API Will Use Artifacts)

In [19]:
def predict_route_efficiency(payload, artifacts):
    """
    Reproduce the exact feature engineering pipeline for a single trip.
    This is the core logic your Flask API will use.
    
    Args:
        payload: dict with raw trip inputs
        artifacts: dict loaded from model_artifacts.joblib
    
    Returns:
        float: predicted route_efficiency [0, 1]
    """
    from datetime import datetime
    
    # Extract artifacts
    model = artifacts['model']
    kmeans_start = artifacts['kmeans_start']
    kmeans_end = artifacts['kmeans_end']
    vehicle_means = artifacts['vehicle_means']
    global_mean = artifacts['global_mean']
    numeric_features = artifacts['numeric_features']
    dummy_columns = artifacts['dummy_columns']
    bucket_bins = artifacts['bucket_bins']
    bucket_labels = artifacts['bucket_labels']
    eps = artifacts['epsilon']
    
    # 1. Parse timestamp
    ts = pd.to_datetime(payload['timestamp_start'])
    hour_of_day = ts.hour
    day_of_week = ts.dayofweek
    
    # 2. Geographic features
    lat1, lon1 = payload['lat_start'], payload['lon_start']
    lat2, lon2 = payload['lat_end'], payload['lon_end']
    
    # Haversine
    R = 6371
    lat1_r, lon1_r, lat2_r, lon2_r = np.radians([lat1, lon1, lat2, lon2])
    dlat = lat2_r - lat1_r
    dlon = lon2_r - lon1_r
    a = np.sin(dlat/2)**2 + np.cos(lat1_r) * np.cos(lat2_r) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    haversine_km = R * c
    
    trip_dist = payload['Trip_Distance_km']
    detour_ratio = trip_dist / (haversine_km + eps)
    directness_eff = np.clip(haversine_km / (trip_dist + eps), 0, 1)
    
    # Bearing
    x = np.sin(dlon) * np.cos(lat2_r)
    y = np.cos(lat1_r) * np.sin(lat2_r) - np.sin(lat1_r) * np.cos(lat2_r) * np.cos(dlon)
    bearing = np.degrees(np.arctan2(x, y))
    
    delta_lat = lat2 - lat1
    delta_lon = lon2 - lon1
    
    # 3. Time features (cyclical)
    hour_sin = np.sin(2 * np.pi * hour_of_day / 24)
    hour_cos = np.cos(2 * np.pi * hour_of_day / 24)
    dow_sin = np.sin(2 * np.pi * day_of_week / 7)
    dow_cos = np.cos(2 * np.pi * day_of_week / 7)
    
    # 4. Operational features
    trip_dur = payload['Trip_Duration_min']
    idle_time = payload['Idle_Time_min']
    moving_time = payload['Moving_Time_min']
    
    idle_ratio = idle_time / (trip_dur + eps)
    moving_ratio = moving_time / (trip_dur + eps)
    idle_per_km = idle_time / (trip_dist + eps)
    
    # Distance bucket
    distance_bucket = pd.cut([trip_dist], bins=bucket_bins, labels=bucket_labels)[0]
    
    # 5. Vehicle encoding
    vehicle_id = payload['vehicle_id']
    vehicle_id_encoded = vehicle_means.get(vehicle_id, global_mean)
    
    # 6. Zone clustering
    zone_start_id = kmeans_start.predict([[lat1, lon1]])[0]
    zone_end_id = kmeans_end.predict([[lat2, lon2]])[0]
    
    # 7. Build feature dict (numeric features)
    feature_dict = {
        'Trip_Distance_km': trip_dist,
        'Trip_Duration_min': trip_dur,
        'Avg_Speed': payload['Avg_Speed'],
        'Idle_Time_min': idle_time,
        'Moving_Time_min': moving_time,
        'Idle_Percentage': payload['Idle_Percentage'],
        'haversine_km': haversine_km,
        'detour_ratio': detour_ratio,
        'bearing': bearing,
        'delta_lat': delta_lat,
        'delta_lon': delta_lon,
        'lat_start': lat1,
        'lon_start': lon1,
        'lat_end': lat2,
        'lon_end': lon2,
        'hour_sin': hour_sin,
        'hour_cos': hour_cos,
        'dow_sin': dow_sin,
        'dow_cos': dow_cos,
        'idle_ratio': idle_ratio,
        'moving_ratio': moving_ratio,
        'idle_per_km': idle_per_km,
        'Age_of_Truck_months': payload['Age_of_Truck_months'],
        'vehicle_id_encoded': vehicle_id_encoded
    }
    
    # 8. One-hot encode categorical (zones + distance_bucket)
    categorical_dict = {
        'zone_start_id': zone_start_id,
        'zone_end_id': zone_end_id,
        'distance_bucket': distance_bucket
    }
    
    # Create temp dataframe for get_dummies
    temp_df = pd.DataFrame([categorical_dict])
    encoded = pd.get_dummies(temp_df, columns=['zone_start_id', 'zone_end_id', 'distance_bucket'],
                            prefix=['zone_start_id', 'zone_end_id', 'distance_bucket'])
    
    # Align to training dummy columns
    for col in dummy_columns:
        if col not in encoded.columns:
            encoded[col] = 0
    encoded = encoded[dummy_columns]
    
    # 9. Combine numeric + categorical in exact training order
    numeric_df = pd.DataFrame([feature_dict])[numeric_features]
    X = pd.concat([numeric_df.reset_index(drop=True), encoded.reset_index(drop=True)], axis=1)
    
    # 10. Predict
    pred = model.predict(X)[0]
    return float(np.clip(pred, 0, 1))


# TEST THE INFERENCE FUNCTION
print("=" * 80)
print("INFERENCE FUNCTION TEST")
print("=" * 80)

# Use a real sample from test set
sample_idx = 5
sample = test_data.iloc[sample_idx]

# Build payload from raw inputs (simulating API call)
test_payload = {
    'vehicle_id': sample['vehicle_id'],
    'timestamp_start': sample['timestamp_start'].isoformat(),
    'lat_start': sample['lat_start'],
    'lon_start': sample['lon_start'],
    'lat_end': sample['lat_end'],
    'lon_end': sample['lon_end'],
    'Trip_Distance_km': sample['Trip_Distance_km'],
    'Trip_Duration_min': sample['Trip_Duration_min'],
    'Idle_Time_min': sample['Idle_Time_min'],
    'Moving_Time_min': sample['Moving_Time_min'],
    'Avg_Speed': sample['Avg_Speed'],
    'Idle_Percentage': sample['Idle_Percentage'],
    'Age_of_Truck_months': sample['Age_of_Truck_months']
}

# Predict using inference function
predicted = predict_route_efficiency(test_payload, loaded_artifacts)

print(f"Sample Index: {sample_idx}")
print(f"Actual route_efficiency: {sample['route_efficiency']:.4f}")
print(f"Predicted (via inference function): {predicted:.4f}")
print(f"Absolute error: {abs(predicted - sample['route_efficiency']):.4f}")
print("=" * 80)
print("\n✓ Inference function works correctly!")
print("✓ Ready to integrate into Flask API")

INFERENCE FUNCTION TEST
Sample Index: 5
Actual route_efficiency: 0.2001
Predicted (via inference function): 0.2003
Absolute error: 0.0003

✓ Inference function works correctly!
✓ Ready to integrate into Flask API
